# Templated Policies

Templated policies are very useful for a wide variety of scenarios. First and foremost, they allow to keep your overall policy count much smaller, and thus make it easier to keep an overview of the configured permissions in your Vault. Another very important use case is that components should not be allowed to create policies on the fly. Unless done with absolute care, this can quickly leads to situations where components can grant themselves root access. In such scenarios templated policies are very useful.

### Starting Vault

Let us start a Vault server. This will run Vault in the background and push the logs to `/tmp/vault.log`. If at any point in time the Vault crashes, this command will need to be used again to re-launch the Vault server.

For now, do not worry about the configuration with which we are starting the server. This will be covered in a separate module.

In [ ]:
nohup bash -c '
  vault server -dev -dev-root-token-id=root-token
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Login to Vault

Let us login to Vault. We started a development version vor simplicity.

In [ ]:
export VAULT_ADDR="http://127.0.0.1:8200"
vault login root-token

### Configuration

Let us setup some stuff that will be useful later: a KV secrets engine and some credentials.

In [ ]:
vault secrets enable -path=kv -version=2 kv
vault kv put -mount=kv teams/cloud-operations/app/portal-frontend/stage/prod/api-token token=85bbddeb-0a81-4cb3-8173-5d6cd414a7a6
vault kv put -mount=kv teams/cloud-operations/app/portal-frontend/stage/test/api-token token=dbe44cad-e3c9-47fd-a725-a08ed0d3d1b8
vault kv put -mount=kv teams/cloud-operations/app/backend/stage/prod/database uri=mariadb.com username=dedicated password=super-secret
vault kv put -mount=kv teams/cloud-operations/app/backend/stage/test/database uri=mariadb.com username=admin password=root
vault kv put -mount=kv teams/cloud-operations/app/backend/stage/prod/ldap-bind uri=ldaps://domain.controller:636 binddn=s-account password=also-super-secret
vault kv put -mount=kv teams/cloud-operations/app/backend/stage/test/ldap-bind uri=ldaps://domain.controller:636 binddn=s-account-testing password=not-super-secret

### Path Structure

Note that we explicitly chose a path structure similar to a REST API:

```
teams/<team>/app/<app>/stage/<stage>/<credential>
```

This is quite a critical point that should not be ignored. Having a correct path structure is the main foundation to enable proper usage of templated policies. The exact structure that you might want to use is highly dependent on how you want to cut permissions, and what the use cases will be. Moreover, it should be noted that the structure should not only consider the paths in the KV secrets engine, but of the entirety of the set of secrets engines. In this example we will only use the KV engine for simplicity, but in reality this would also include other engines such as PKI, dynamic credentials, etc.

### Creating a Simple Templated Policy

The simplest form of templating in policies comes in the form of globbing characters: `*` and `+`. The `*` (asterisk) means any set of path segments and can only be used at the very end of a policy path. The `+` (plus) means a single path segment and can be used anywhere in the path string of a policy. Thus for instance the following policy:

```hcl
path "kv/data/teams/cloud-operations/*" {
  capabilities = [ "read" ]
}
```

grants read access to all credentials owned by the `cloud-operations` team. On the other hand:

```hcl
path "kv/data/teams/cloud-operations/app/portal-frontend/stage/+/api-token" {
  capabilities = [ "read" ]
}
```

Grants access to the `api-token` credentials for all stages. Note that the `+` can also be used as the last element of the path to ensure only a single path segment is granted and not any path below the provided root.

Create three templated polices that:

1. Grant read access to any credentials within the `backend` application of the `cloud-operation` team.
2. Grant read access to any `test` credential within the `cloud-operation` team, regardless of the application.
3. Grant read access to any credentials of any app called `backend` regardless of what team owns the application.

In [ ]:
vault policy write read-cloud-operations-backend-secrets -<<-EOF
path "kv/data/..." {
  capabilities = [ "read" ]
}
EOF

In [ ]:
vault policy write read-cloud-operations-test-secrets -<<-EOF
path "kv/data/..." {
  capabilities = [ "read" ]
}
EOF

In [ ]:
vault policy write read-backend-secrets -<<-EOF
path "kv/data/..." {
  capabilities = [ "read" ]
}
EOF

### Testing the Policies

Let us test the policies above.

In [ ]:
# This should PASS
vault login root-token > /dev/null
vault login "$(vault token create -policy=read-cloud-operations-backend-secrets -field=token)" > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/backend/stage/prod/ldap-bind > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/backend/stage/test/ldap-bind > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/backend/stage/test/database > /dev/null

In [ ]:
# This should FAIL
vault login root-token > /dev/null
vault login "$(vault token create -policy=read-cloud-operations-backend-secrets -field=token)" > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/portal-frontend/stage/test/api-token > /dev/null

In [ ]:
# This should PASS
vault login root-token > /dev/null
vault login "$(vault token create -policy=read-cloud-operations-test-secrets -field=token)" > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/backend/stage/test/ldap-bind > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/backend/stage/test/database > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/portal-frontend/stage/test/api-token > /dev/null

In [ ]:
# This should FAIL
vault login root-token > /dev/null
vault login "$(vault token create -policy=read-cloud-operations-test-secrets -field=token)" > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/portal-frontend/stage/prod/api-token > /dev/null

In [ ]:
# This should PASS
vault login root-token > /dev/null
vault login "$(vault token create -policy=read-backend-secrets -field=token)" > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/backend/stage/test/ldap-bind > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/backend/stage/prod/database > /dev/null

In [ ]:
# This should FAIL with "No value found at kv/data/teams/does-not-exist/app/backend/something"
vault login root-token > /dev/null
vault login "$(vault token create -policy=read-backend-secrets -field=token)" > /dev/null
vault kv get -mount=kv teams/does-not-exist/app/backend/something > /dev/null

In [ ]:
# This should FAIL
vault login root-token > /dev/null
vault login "$(vault token create -policy=read-backend-secrets -field=token)" > /dev/null
vault kv get -mount=kv teams/cloud-operations/app/portal-frontend/stage/test/api-token > /dev/null

### Real Templating

The above is nice, but won't get you far when you start having more complex setups. You will want to dynamically provide access based on who/what logs in to Vault. We will do this using the userpass authentication engine. In practice, you would not use the userpass engine but rather an OIDC or LDAP integration for human authentication. Nonetheless, the concepts apply exactly the same way.

Let us create two users for userpass that have different metadata attached to them. In practice this would be injected automatically. For instance, you might want to grant access to a Kubernetes ServiceAccount based on what namespace it is in (since you might cut namespaces based on teams). In such a case, the namespace name will be automatically provided as metadata to the identity when logging in using the Kubernetes authentication method.

Let's prepare the userpass method.

In [ ]:
kind create cluster --name templated-policies --kubeconfig kube.yaml

In [ ]:
# update kube config if needed
[ -f /.dockerenv ] && sed -i 's/127.0.0.1.*$/templated-policies-control-plane:6443/' kube.yaml
export KUBECONFIG=./kube.yaml
kubectl cluster-info --context kind-templated-policies

In [ ]:
kubectl get --raw /openid/v1/jwks

In [ ]:
docker ps

## Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault.log
rm /tmp/vault.pid
kind delete cluster --name templated-policies